In [1]:
#import usual packages for data manipulation
import pandas as pd
import numpy as np

#import usual packages for data visualization
import matplotlib.pyplot as plt
import seaborn as sns

#import yfinance to get stock data
import yfinance as yf

#import quantstats to get some statistics on the stock data
import quantstats as qs

#import statistical packages
import statsmodels.api as sm
import scipy.stats as scs

#import datetime to get the current date
import datetime as dt

#set the style of the plots
plt.style.use('seaborn-v0_8-whitegrid')

In [4]:
#Arvind's API key
my_key = 'DJ3QKKTFZ5J298QY'

#from alpha_vantage import EconIndicators
from alpha_vantage.econindicators import EconIndicators

In [37]:
#instantiate the econindicators class
ei = EconIndicators(key=my_key, output_format='pandas',indexing_type='integer')

In [38]:
#get following date: quarterly gdp percap, monthly treasury yield (10yrs), monthly unemployment rate, monthly inflation rate, monthly consumer price index
gdp_percap, _ = ei.get_real_gdp_per_capita()
treasury_yield, _ = ei.get_treasury_yield(interval='monthly',maturity='10year')
federal_funds_rate, _ = ei.get_ffr(interval='monthly')
consumer_price_index, _ = ei.get_cpi(interval='monthly')
inflation, _ = ei.get_inflation()
unemployment_rate, _ = ei.get_unemployment()

In [62]:
#loop through all dataframes above and do the following:
#1. drop the index column
#2. reset the index with drop=True
#3. convert the date column to datetime and set it as the index
#4. drop all rows corresponding to years before 2014

for df in [gdp_percap, treasury_yield, federal_funds_rate, consumer_price_index, inflation, unemployment_rate]:
    df.drop(columns='index',inplace=True)
    df.reset_index(drop=True,inplace=True)
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date',inplace=True)
    df.drop(df[df.index.year < 2014].index,inplace=True)

#merge all dataframes into one
df_econindicators = pd.concat([gdp_percap, treasury_yield, federal_funds_rate, consumer_price_index, inflation, unemployment_rate],axis=1)
#rename columns
df_econindicators.columns = ['gdp_percap','treasury_yield','federal_funds_rate','consumer_price_index','inflation','unemployment_rate']
#forward fill missing values
df_econindicators = df_econindicators.ffill()
#save to csv
df_econindicators.to_csv('../data/econindicators.csv')

In [96]:
df_econindicators

,gdp_percap,treasury_yield,federal_funds_rate,consumer_price_index,inflation,unemployment_rate
date,,,,,,
2014-01-01,56345.0,2.86,0.07,233.916,1.62222297740827,6.6
2014-02-01,56345.0,2.71,0.07,234.781,1.62222297740827,6.7
2014-03-01,56345.0,2.72,0.08,236.293,1.62222297740827,6.7
2014-04-01,56968.0,2.71,0.09,237.072,1.62222297740827,6.2
2014-05-01,56968.0,2.56,0.09,237.900,1.62222297740827,6.3
...,...,...,...,...,...,...
2024-03-01,67672.0,4.21,5.33,312.332,4.11633838374488,3.8
2024-04-01,68070.0,4.54,5.33,313.548,4.11633838374488,3.9
2024-05-01,68070.0,4.48,5.33,314.069,4.11633838374488,4.0


In [148]:
#import company financials data with the following code:

from alpha_vantage.fundamentaldata import FundamentalData
fd = FundamentalData(key=my_key)

#choose a ticker
ticker = 'MSFT'

#get quarterly earnings for "ticker"
df_earnings, _ = fd.get_earnings_quarterly(ticker)

#get quarterly balance sheet for "ticker"
df_balance, _ = fd.get_balance_sheet_quarterly(ticker)

#get quarterly cash flow for "ticker"
df_cashflow, _ = fd.get_cash_flow_quarterly(ticker)

#get quarterly income statement for "ticker"
df_income, _ = fd.get_income_statement_quarterly(ticker)

,fiscalDateEnding,reportedDate,reportedEPS,estimatedEPS,surprise,surprisePercentage,reportTime
0,2024-06-30,2024-07-30,2.95,2.93,0.02,0.6826,post-market
1,2024-03-31,2024-04-25,2.94,2.82,0.12,4.2553,post-market
2,2023-12-31,2024-01-30,2.93,2.78,0.15,5.3957,post-market
3,2023-09-30,2023-10-24,2.99,2.65,0.34,12.8302,post-market
4,2023-06-30,2023-07-25,2.69,2.55,0.14,5.4902,post-market
...,...,...,...,...,...,...,...
109,1997-03-31,1997-04-17,0.1,0.08,0.02,25,pre-market
110,1996-12-31,1997-01-17,0.07,0.06,0.01,16.6667,pre-market
111,1996-09-30,1996-10-21,0.06,0.06,0,0,pre-market
112,1996-06-30,1996-07-22,0.05,0.05,0,0,pre-market


In [135]:
#read in the four aapl financials csv's contained in the data folder
df_income = pd.read_csv('../data/aapl_income_statement.csv')
df_balance = pd.read_csv('../data/aapl_balance_sheet.csv')
df_cashflow = pd.read_csv('../data/aapl_cashflow.csv')
df_earnings = pd.read_csv('../data/aapl_earnings.csv')

#for each of the four dataframes above, do the following:
# drop the 'Unnamed: 0' column
# drop the 'reportedCurrency' column
# drop all columns with at least 10 missing values
# convert the 'fidcalDateEnding' column to datetime and set it as the index.
#rename the 'fiscalDateEnding' column to 'date'
#drop all rows corresponding to years before 2014
#if any dates are duplicated, keep the first one and drop the rest
for df in [df_income, df_balance, df_cashflow, df_earnings]:
    df.drop(columns='Unnamed: 0',inplace=True)
    df.drop_duplicates(subset='fiscalDateEnding',keep='first',inplace=True)
    df.dropna(axis=1,thresh=df.shape[0]-10,inplace=True)
    df['date'] = pd.to_datetime(df['fiscalDateEnding'])
    df.drop(columns='fiscalDateEnding',inplace=True)
    df.set_index('date',inplace=True)
    df.drop(df[df.index.year < 2014].index,inplace=True)

In [136]:
financials = {'income':df_income, 
              'balance':df_balance, 
              'cashflow':df_cashflow, 
              'earnings':df_earnings}
for i, df in financials.items():
    print(i)
    print(len(df))


income
42
balance
42
cashflow
42
earnings
42


In [141]:
df_financials = pd.concat([df_income, df_balance, df_cashflow, df_earnings],axis=1,join='inner')

In [144]:
#drop all duplicate columns
df_financials = df_financials.loc[:,~df_financials.columns.duplicated()]
df_financials

,reportedCurrency,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,interestExpense,...,cashflowFromFinancing,paymentsForRepurchaseOfCommonStock,paymentsForRepurchaseOfEquity,dividendPayout,proceedsFromIssuanceOfCommonStock,proceedsFromRepurchaseOfEquity,changeInCashAndCashEquivalents,reportedDate,reportedEPS,reportTime
date,,,,,,,,,,,,,,,,,,,,,
2024-06-30,USD,39678000000,85777000000,52419000000,46099000000,25352000000,6320000000,8006000000,14326000000,NaN,...,-36017000000,2.652200e+10,2.652200e+10,3.895000e+09,NaN,-26522000000,NaN,2024-08-01,1.4000,post-market
2024-03-31,USD,42271000000,90753000000,54950000000,48482000000,27900000000,6468000000,7903000000,14371000000,NaN,...,-30433000000,2.320500e+10,2.320500e+10,3.710000e+09,NaN,-23205000000,NaN,2024-05-02,1.5300,post-market
2023-12-31,USD,54855000000,119575000000,71506000000,64720000000,40373000000,6786000000,7696000000,14482000000,1.002000e+09,...,-30585000000,2.013900e+10,2.013900e+10,3.825000e+09,NaN,-20139000000,NaN,2024-02-01,2.1800,post-market
2023-09-30,USD,40427000000,88496000000,55222000000,49071000000,26969000000,6151000000,7307000000,13458000000,1.002000e+09,...,-23153000000,2.100300e+10,2.100300e+10,3.758000e+09,0.0,-21003000000,8.390000e+08,2023-11-02,1.4600,post-market
2023-06-30,USD,36413000000,80799000000,51357000000,45384000000,22998000000,5973000000,7442000000,13415000000,9.980000e+08,...,-24048000000,1.747800e+10,0.000000e+00,3.849000e+09,0.0,0,2.769000e+09,2023-08-03,1.2600,post-market
2023-03-31,USD,41976000000,94836000000,59061000000,52860000000,28318000000,6201000000,7457000000,13658000000,9.300000e+08,...,-25724000000,1.959400e+10,1.959400e+10,3.650000e+09,NaN,-19594000000,5.155000e+09,2023-05-04,1.5200,post-market
2022-12-31,USD,50332000000,116151000000,73429000000,66822000000,36016000000,6607000000,7709000000,14316000000,1.003000e+09,...,-35563000000,1.947500e+10,1.947500e+10,3.768000e+09,0.0,-19475000000,-3.003000e+09,2023-02-02,1.8800,post-market
2022-09-30,USD,38095000000,89319000000,58491000000,52051000000,24894000000,6440000000,6761000000,13201000000,8.270000e+08,...,-26794000000,2.442800e+10,2.442800e+10,3.703000e+09,NaN,-24428000000,-3.884000e+09,2022-10-27,1.2900,post-market
2022-06-30,USD,35885000000,82240000000,53086000000,47074000000,23076000000,6012000000,6797000000,12809000000,7.190000e+08,...,-27445000000,2.186500e+10,2.186500e+10,3.811000e+09,NaN,-21865000000,1.944200e+10,2022-07-28,1.2000,post-market


In [147]:
print(df_financials.head(1))

           reportedCurrency  grossProfit  totalRevenue  costOfRevenue  \
date                                                                    
2024-06-30              USD  39678000000   85777000000    52419000000   

            costofGoodsAndServicesSold  operatingIncome  \
date                                                      
2024-06-30                 46099000000      25352000000   

            sellingGeneralAndAdministrative  researchAndDevelopment  \
date                                                                  
2024-06-30                       6320000000              8006000000   

            operatingExpenses  interestExpense  ...  cashflowFromFinancing  \
date                                            ...                          
2024-06-30        14326000000              NaN  ...           -36017000000   

            paymentsForRepurchaseOfCommonStock  paymentsForRepurchaseOfEquity  \
date                                                                    

In [145]:
list(df_financials.columns)

['reportedCurrency',
 'grossProfit',
 'totalRevenue',
 'costOfRevenue',
 'costofGoodsAndServicesSold',
 'operatingIncome',
 'sellingGeneralAndAdministrative',
 'researchAndDevelopment',
 'operatingExpenses',
 'interestExpense',
 'otherNonOperatingIncome',
 'depreciationAndAmortization',
 'incomeBeforeTax',
 'incomeTaxExpense',
 'netIncomeFromContinuingOperations',
 'comprehensiveIncomeNetOfTax',
 'ebit',
 'ebitda',
 'netIncome',
 'totalAssets',
 'totalCurrentAssets',
 'cashAndCashEquivalentsAtCarryingValue',
 'cashAndShortTermInvestments',
 'inventory',
 'currentNetReceivables',
 'totalNonCurrentAssets',
 'propertyPlantEquipment',
 'accumulatedDepreciationAmortizationPPE',
 'longTermInvestments',
 'shortTermInvestments',
 'otherCurrentAssets',
 'otherNonCurrentAssets',
 'totalLiabilities',
 'totalCurrentLiabilities',
 'currentAccountsPayable',
 'totalNonCurrentLiabilities',
 'otherCurrentLiabilities',
 'otherNonCurrentLiabilities',
 'totalShareholderEquity',
 'retainedEarnings',
 'comm

In [100]:
#merge the four dataframes into one
df_financials = pd.concat([df_income, df_balance, df_cashflow, df_earnings],axis=1,join='outer')

InvalidIndexError: Reindexing only valid with uniquely valued Index objects